# Confidence Interval Test for Confined Aquifers
**Synthetic data**

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ttim import *

Set basic parameters for the model:

In [ ]:
H = 7 #aquifer thickness
k = 70 #hydraulic conductivity
S = 1e-4 #specific storage
Q = 788 #constant discharge
d1 = 30 #observation well 1
d2 = 90 #observation well 2 (positions same as for Oude Korendijk)

Load data of test site 'Oude Korendijk':

In [ ]:
data1 = np.loadtxt('data/piezometer_h30.txt', skiprows = 1)
t = data1[:, 0] / 60 / 24 # convert min to days

Create conceptual model:

In [ ]:
ml = ModelMaq(kaq=70, z =[-18, -25], Saq=1e-4, tmin=1e-5, tmax=1)
w = Well(ml, xw=0, yw=0, rw=0.1, tsandQ=[(0, 788)])
ml.solve(silent='True')
h1 = ml.head(d1, 0, t)
h2 = ml.head(d2, 0, t)

Add noises:

In [ ]:
np.savetxt('data/syn_30_0.0.txt', h1[0])
np.savetxt('data/syn_90_0.0.txt', h2[0])
#print(h2[0])

In [ ]:
np.random.seed(5)
he12 = h1[0] - np.random.randn(len(t)) * 0.02
he22 = h2[0] - np.random.randn(len(t)) * 0.02
np.savetxt('data/syn_p30_0.02.txt', he12)
np.savetxt('data/syn_p90_0.02.txt', he22)

In [ ]:
np.random.seed(4)
he15 = h1[0] - np.random.randn(len(t)) * 0.05
he25 = h2[0] - np.random.randn(len(t)) * 0.05
np.savetxt('data/syn_p30_0.05.txt', he15)
np.savetxt('data/syn_p90_0.05.txt', he25)

In [ ]:
plt.figure(figsize=(8, 5))
plt.semilogx(t, he12, '.', label='obs at 30 m with sig=0.02')
plt.semilogx(t, h1[0], label='ttim at 30 m')
plt.semilogx(t, he22, '.', label='obs at 90 m with sig=0.02')
plt.semilogx(t, h2[0], label='ttim at 90 m')
plt.legend()
plt.xlabel('time (d)')
plt.ylabel('drawdown (m)');

In [ ]:
plt.figure(figsize=(8, 5))
plt.semilogx(t, he15, '.', label='obs at 30 m with sig=0.05')
plt.semilogx(t, h1[0], label='ttim at 30 m')
plt.semilogx(t, he25, '.', label='obs at 90 m with sig=0.05')
plt.semilogx(t, h2[0], label='ttim at 90 m')
plt.legend()
plt.xlabel('time (d)')
plt.ylabel('drawdown (m)');

#### Test if TTim finds the parameters back

Calibrate with two datasets respectively (0.02):

In [ ]:
ca23 = Calibrate(ml)
ca23.set_parameter(name='kaq0', initial=10)
ca23.set_parameter(name='Saq0', initial=1e-3)
ca23.series(name='obs1', x=d1, y=0, t=t, h=he12, layer=0)
ca23.fit(report=True)
display(ca23.parameters)

In [ ]:
print('rmse:', ca23.rmse())
h123 = ml.head(d1, 0, t)
h223 = ml.head(d2, 0 ,t)
plt.figure(figsize = (8, 5))
plt.semilogx(t, he12, '.', label='obs at 30 m with sig=0.02')
plt.semilogx(t, h123[0], label='ttim at 30 m')
plt.semilogx(t, he22, '.', label='obs at 90 m with sig=0.02')
plt.semilogx(t, h223[0], label='ttim at 90 m')
plt.xlabel('time (d)')
plt.ylabel('drawdown (m)')
plt.title('ttim analysis with synthetic data, sig=0.02 errors at 30 m.')
plt.legend();

In [ ]:
ca29 = Calibrate(ml)
ca29.set_parameter(name='kaq0', initial=10)
ca29.set_parameter(name='Saq0', initial=1e-3)
ca29.series(name='obs2', x=d2, y=0, t=t, h=he22, layer=0)
ca29.fit(report=True)
display(ca29.parameters)

In [ ]:
print('rmse:', ca29.rmse())
h129 = ml.head(d1, 0, t)
h229 = ml.head(d2, 0, t)
plt.figure(figsize = (8, 5))
plt.semilogx(t, he15, '.', label='obs at 30 m with sig=0.02')
plt.semilogx(t, h129[0], label='ttim at 30 m')
plt.semilogx(t, he25, '.', label='obs at 90 m with sig=0.02')
plt.semilogx(t, h229[0], label='ttim at 90 m')
plt.legend()
plt.xlabel('time (d)')
plt.ylabel('drawdown (m)');
plt.title('ttim analysis with synthetic data, sig=0.02 errors at 90 m.')
plt.legend(loc = 'best');

#### Calibrate with two datasets simultaneously

Drawdown without errors:

In [ ]:
ca0 = Calibrate(ml)
ca0.set_parameter(name='kaq0', initial=10)
ca0.set_parameter(name='Saq0', initial=1e-3)
ca0.series(name='obs1', x=d1, y=0, t=t, h=h1[0], layer=0)
ca0.series(name='obs2', x=d2, y=0, t=t, h=h2[0], layer=0)
ca0.fit(report=True)
display(ca0.parameters)

In [ ]:
print('rmse:', ca0.rmse())
h1n = ml.head(d1, 0, t)
h2n = ml.head(d2, 0, t)
plt.figure(figsize = (7, 4))
plt.semilogx(t, h1[0], 'b.', label='obs at 30 m no errors')
plt.semilogx(t, h1n[0], color = 'r', label = 'ttim at 30 m')
plt.semilogx(t, h2[0], 'g.', label='obs at 90 m no errors')
plt.semilogx(t, h2n[0], color='orange', label = 'ttim at 90 m')
plt.xlabel('time (d)')
plt.ylabel('drawdown (m)')
plt.title('ttim analysis with synthetic data without errors.')
plt.legend();

Drawdowns with errors with $\sigma=0.02$.

In [ ]:
ca2 = Calibrate(ml)
ca2.set_parameter(name='kaq0', initial=10)
ca2.set_parameter(name='Saq0', initial=1e-3)
ca2.series(name='obs1', x=d1, y=0, t=t, h=he12, layer=0)
ca2.series(name='obs2', x=d2, y=0, t=t, h=he22, layer=0)
ca2.fit()
display(ca2.parameters)

In [ ]:
print('rmse:', ca2.rmse())
h12 = ml.head(d1, 0, t)
h22 = ml.head(d2, 0, t)
plt.figure(figsize = (8, 5))
plt.semilogx(t, he12, 'b.', label='obs at 30 m, sig=0.02')
plt.semilogx(t, h12[0], color = 'r', label = 'ttim at 30 m')
plt.semilogx(t, he22, 'g.', label='obs at 90 m, sig=0.02')
plt.semilogx(t, h22[0], color='orange', label = 'ttim at 90 m')
plt.xlabel('time (d)')
plt.ylabel('drawdown (m)')
plt.title('ttim analysis with synthetic data and errors with sig=0.02')
plt.legend();

Drawdowns with errors with $\sigma=0.05$.

In [ ]:
ca5 = Calibrate(ml)
ca5.set_parameter(name='kaq0', initial=10)
ca5.set_parameter(name='Saq0', initial=1e-3)
ca5.series(name='obs1', x=d1, y=0, t=t, h=he15, layer=0)
ca5.series(name='obs2', x=d2, y=0, t=t, h=he25, layer=0)
ca5.fit()
display(ca5.parameters)

In [ ]:
print('rmse:', ca5.rmse())
h15 = ml.head(d1, 0, t)
h25 = ml.head(d2, 0, t)
plt.figure(figsize=(8, 5))
plt.semilogx(t, he15, 'b.', label='obs at 30 m')
plt.semilogx(t, h15[0], color = 'r', label = 'ttim at 30 m')
plt.semilogx(t, he25, 'g.', label='obs at 90 m')
plt.semilogx(t, h25[0], color='orange', label = 'ttim at 90 m')
plt.xlabel('time (d)')
plt.ylabel('drawdown (m)')
plt.title('ttim analysis with synthetic data and errors with sig=0.05')
plt.legend();